In [1]:
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully!")

# ============================================================================
# STEP 3: CONFIGURATION
# ============================================================================

# Change this if your file has a different name or location
CSV_FILE = 'stress_detection_realistic_1500_rows.csv'

# Check if file exists
if not os.path.exists(CSV_FILE):
    print(f"\n❌ ERROR: File not found: {CSV_FILE}")
    print(f"Current directory: {os.getcwd()}")
    print("\nFiles in current directory:")
    for f in os.listdir('.')[:20]:
        print(f"  - {f}")
    raise FileNotFoundError(f"Please put {CSV_FILE} in the current directory")

print(f"✓ Found file: {CSV_FILE}")

# ============================================================================
# STEP 4: LOAD AND PREPARE DATA
# ============================================================================

print("\n" + "="*70)
print("LOADING DATA")
print("="*70)

# Load data
df = pd.read_csv(CSV_FILE)
print(f"✓ Loaded {len(df)} samples with {df.shape[1]} columns")

# Show first few rows
print("\nFirst 5 rows:")
print(df.head())

# Show stress distribution
print("\nStress Level Distribution:")
for level in [0, 1, 2]:
    count = (df['stress_level'] == level).sum()
    level_name = ['Low', 'Medium', 'High'][level]
    print(f"  {level_name} (level {level}): {count} samples ({count/len(df)*100:.1f}%)")

# Separate features and target
X = df.drop('stress_level', axis=1)
y = df['stress_level']
feature_names = list(X.columns)

print(f"\nFeatures ({len(feature_names)}): {feature_names}")

✓ All libraries imported successfully!
✓ Found file: stress_detection_realistic_1500_rows.csv

LOADING DATA
✓ Loaded 1500 samples with 9 columns

First 5 rows:
   heart_rate    hrv   spo2  systolic_bp  diastolic_bp  body_temp  \
0       77.08  81.73  94.53       101.80         79.18      36.89   
1       78.57  46.31  95.47       121.38         80.30      37.27   
2       76.46  56.02  90.85       127.18        102.17      36.74   
3       92.17  38.03  92.65       135.62         99.26      37.13   
4       76.50  38.17  92.31       119.21         89.41      36.98   

   sleep_hours  blink_rate  stress_level  
0         8.30       20.08             0  
1         4.30       27.80             1  
2         3.03       13.01             1  
3         2.43       30.36             1  
4         5.55       11.98             1  

Stress Level Distribution:
  Low (level 0): 500 samples (33.3%)
  Medium (level 1): 500 samples (33.3%)
  High (level 2): 500 samples (33.3%)

Features (8): ['heart_r

In [2]:
print("\n" + "="*70)
print("LOADING DATA")
print("="*70)

# Load data
df = pd.read_csv(CSV_FILE)
print(f"✓ Loaded {len(df)} samples with {df.shape[1]} columns")

# Show first few rows
print("\nFirst 5 rows:")
print(df.head())

# Show stress distribution
print("\nStress Level Distribution:")
for level in [0, 1, 2]:
    count = (df['stress_level'] == level).sum()
    level_name = ['Low', 'Medium', 'High'][level]
    print(f"  {level_name} (level {level}): {count} samples ({count/len(df)*100:.1f}%)")

# Separate features and target
X = df.drop('stress_level', axis=1)
y = df['stress_level']
feature_names = list(X.columns)

print(f"\nFeatures ({len(feature_names)}): {feature_names}")

# ============================================================================
# STEP 5: SPLIT AND SCALE DATA
# ============================================================================

print("\n" + "="*70)
print("PREPARING DATA")
print("="*70)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✓ Features scaled")


LOADING DATA
✓ Loaded 1500 samples with 9 columns

First 5 rows:
   heart_rate    hrv   spo2  systolic_bp  diastolic_bp  body_temp  \
0       77.08  81.73  94.53       101.80         79.18      36.89   
1       78.57  46.31  95.47       121.38         80.30      37.27   
2       76.46  56.02  90.85       127.18        102.17      36.74   
3       92.17  38.03  92.65       135.62         99.26      37.13   
4       76.50  38.17  92.31       119.21         89.41      36.98   

   sleep_hours  blink_rate  stress_level  
0         8.30       20.08             0  
1         4.30       27.80             1  
2         3.03       13.01             1  
3         2.43       30.36             1  
4         5.55       11.98             1  

Stress Level Distribution:
  Low (level 0): 500 samples (33.3%)
  Medium (level 1): 500 samples (33.3%)
  High (level 2): 500 samples (33.3%)

Features (8): ['heart_rate', 'hrv', 'spo2', 'systolic_bp', 'diastolic_bp', 'body_temp', 'sleep_hours', 'blink_rate']


In [3]:
print("\n" + "="*70)
print("TRAINING MODELS")
print("="*70)

# Define models
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42),
    'ANN': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
}

# Train and evaluate each model
results = {}

for name, model in models.items():
    print(f"\n{name}:")
    
    # Train
    print("  Training...", end=" ")
    model.fit(X_train_scaled, y_train)
    print("✓")
    
    # Test
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Store results
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'y_pred': y_pred
    }
    
    print(f"  Accuracy: {accuracy*100:.2f}%")

# ============================================================================
# STEP 7: FIND BEST MODEL
# ============================================================================

print("\n" + "="*70)
print("RESULTS")
print("="*70)

# Create results table
results_df = pd.DataFrame({
    name: {'Accuracy': res['accuracy']}
    for name, res in results.items()
}).T

print("\nModel Comparison:")
print(results_df.to_string())

# Find best model
best_model_name = results_df['Accuracy'].idxmax()
best_accuracy = results_df['Accuracy'].max()

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   Accuracy: {best_accuracy*100:.2f}%")

# Show confusion matrix for best model
print(f"\nConfusion Matrix ({best_model_name}):")
cm = confusion_matrix(y_test, results[best_model_name]['y_pred'])
print(cm)
print("\n         Predicted")
print("         Low  Med  High")
print(f"Low      {cm[0,0]:3d}  {cm[0,1]:3d}  {cm[0,2]:3d}")
print(f"Medium   {cm[1,0]:3d}  {cm[1,1]:3d}  {cm[1,2]:3d}")
print(f"High     {cm[2,0]:3d}  {cm[2,1]:3d}  {cm[2,2]:3d}")

# Calculate per-class accuracy
print("\nPer-Class Accuracy:")
for i, level_name in enumerate(['Low', 'Medium', 'High']):
    correct = cm[i, i]
    total = cm[i, :].sum()
    acc = correct / total * 100 if total > 0 else 0
    print(f"  {level_name}: {acc:.1f}% ({correct}/{total})")



TRAINING MODELS

Random Forest:
  Training... ✓
  Accuracy: 92.00%

SVM:
  Training... ✓
  Accuracy: 94.33%

ANN:
  Training... ✓
  Accuracy: 92.00%

RESULTS

Model Comparison:
               Accuracy
Random Forest  0.920000
SVM            0.943333
ANN            0.920000

🏆 BEST MODEL: SVM
   Accuracy: 94.33%

Confusion Matrix (SVM):
[[97  3  0]
 [ 3 91  6]
 [ 0  5 95]]

         Predicted
         Low  Med  High
Low       97    3    0
Medium     3   91    6
High       0    5   95

Per-Class Accuracy:
  Low: 97.0% (97/100)
  Medium: 91.0% (91/100)
  High: 95.0% (95/100)


In [4]:
print("\n" + "="*70)
print("SAVING MODELS")
print("="*70)

# Create folder
os.makedirs('stress_models', exist_ok=True)

# Save all models
for name, res in results.items():
    filename = f"stress_models/{name.lower().replace(' ', '_')}.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(res['model'], f)
    print(f"✓ Saved: {filename}")

# Save best model
with open('stress_models/best_model.pkl', 'wb') as f:
    pickle.dump(results[best_model_name]['model'], f)
print(f"✓ Saved: stress_models/best_model.pkl")

# Save scaler
with open('stress_models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print(f"✓ Saved: stress_models/scaler.pkl")

# Save feature names
with open('stress_models/features.pkl', 'wb') as f:
    pickle.dump(feature_names, f)
print(f"✓ Saved: stress_models/features.pkl")



SAVING MODELS
✓ Saved: stress_models/random_forest.pkl
✓ Saved: stress_models/svm.pkl
✓ Saved: stress_models/ann.pkl
✓ Saved: stress_models/best_model.pkl
✓ Saved: stress_models/scaler.pkl
✓ Saved: stress_models/features.pkl


In [7]:
print("\n" + "="*70)
print("CREATING PREDICTION SYSTEM")
print("="*70)

def predict_stress(heart_rate, hrv, spo2, systolic_bp, diastolic_bp, 
                   body_temp, sleep_hours, blink_rate):
    """
    Predict stress level from health parameters
    
    Returns: dict with level (0/1/2), label (Low/Medium/High), and confidence
    """
    # Load model and scaler
    with open('stress_models/best_model.pkl', 'rb') as f:
        model = pickle.load(f)
    with open('stress_models/scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)
    
    # Prepare input
    features = [[heart_rate, hrv, spo2, systolic_bp, diastolic_bp, 
                 body_temp, sleep_hours, blink_rate]]
    features_scaled = scaler.transform(features)
    
    # Predict
    prediction = model.predict(features_scaled)[0]
    probabilities = model.predict_proba(features_scaled)[0]
    confidence = probabilities[prediction]
    
    # Format result
    levels = ['Low', 'Medium', 'High']
    icons = ['🟢', '🟡', '🔴']
    
    return {
        'level': int(prediction),
        'label': levels[prediction],
        'icon': icons[prediction],
        'confidence': float(confidence),
        'probabilities': {
            'Low': float(probabilities[0]),
            'Medium': float(probabilities[1]),
            'High': float(probabilities[2])
        }
    }

print("✓ Prediction function created: predict_stress()")

# ============================================================================
# STEP 10: TEST WITH EXAMPLES
# ============================================================================

print("\n" + "="*70)
print("EXAMPLE PREDICTIONS")
print("="*70)

examples = [
    {
        'name': 'Healthy Person (Low Stress)',
        'params': (70, 75, 98, 110, 75, 36.5, 8, 15)
    },
    {
        'name': 'Moderate Stress',
        'params': (88, 42, 96, 128, 84, 37.3, 5.5, 26)
    },
    {
        'name': 'High Stress',
        'params': (105, 25, 94, 145, 92, 37.8, 3.5, 35)
    }
]

for ex in examples:
    result = predict_stress(*ex['params'])
    print(f"\n{ex['name']}:")
    print(f"  Prediction: {result['icon']} {result['label']}")
    print(f"  Confidence: {result['confidence']:.1%}")

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*70)
print("✨ TRAINING COMPLETE! ✨")
print("="*70)

print(f"""
SUMMARY:
--------
✓ Dataset: {len(df)} samples
✓ Best Model: {best_model_name} ({best_accuracy*100:.2f}% accuracy)
✓ Models saved to: stress_models/
""")


CREATING PREDICTION SYSTEM
✓ Prediction function created: predict_stress()

EXAMPLE PREDICTIONS

Healthy Person (Low Stress):
  Prediction: 🟢 Low
  Confidence: 100.0%

Moderate Stress:
  Prediction: 🟡 Medium
  Confidence: 99.9%

High Stress:
  Prediction: 🔴 High
  Confidence: 100.0%

✨ TRAINING COMPLETE! ✨

SUMMARY:
--------
✓ Dataset: 1500 samples
✓ Best Model: SVM (94.33% accuracy)
✓ Models saved to: stress_models/



In [8]:
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("MODEL VERIFICATION & PREDICTION SYSTEM")
print("="*80)

# ============================================================================
# PART 1: VERIFY MODEL IS TRAINED PROPERLY
# ============================================================================

print("\n" + "="*80)
print("PART 1: VERIFYING MODEL TRAINING")
print("="*80)

def verify_model_training():
    """Check if models are trained properly"""
    
    print("\n1.1 Checking if model files exist...")
    
    required_files = [
        'stress_models/best_model.pkl',
        'stress_models/scaler.pkl',
        'stress_models/features.pkl'
    ]
    
    all_exist = True
    for file in required_files:
        if os.path.exists(file):
            print(f"  ✓ Found: {file}")
        else:
            print(f"  ✗ Missing: {file}")
            all_exist = False
    
    if not all_exist:
        print("\n❌ ERROR: Some model files are missing!")
        print("   Please run the training script first.")
        return False
    
    print("\n1.2 Loading and testing model...")
    
    try:
        # Load model
        with open('stress_models/best_model.pkl', 'rb') as f:
            model = pickle.load(f)
        print("  ✓ Model loaded successfully")
        
        # Load scaler
        with open('stress_models/scaler.pkl', 'rb') as f:
            scaler = pickle.load(f)
        print("  ✓ Scaler loaded successfully")
        
        # Load features
        with open('stress_models/features.pkl', 'rb') as f:
            features = pickle.load(f)
        print(f"  ✓ Features loaded: {features}")
        
        # Test with dummy data
        test_data = [[70, 75, 98, 110, 75, 36.5, 8, 15]]
        test_scaled = scaler.transform(test_data)
        prediction = model.predict(test_scaled)
        probabilities = model.predict_proba(test_scaled)
        
        print(f"\n  ✓ Model test successful!")
        print(f"    Test prediction: {prediction[0]}")
        print(f"    Test probabilities: {probabilities[0]}")
        
        return True
        
    except Exception as e:
        print(f"\n❌ ERROR loading model: {e}")
        return False

# Run verification
model_ok = verify_model_training()

if not model_ok:
    print("\n⚠️ Please train the model first before proceeding!")
    exit()

MODEL VERIFICATION & PREDICTION SYSTEM

PART 1: VERIFYING MODEL TRAINING

1.1 Checking if model files exist...
  ✓ Found: stress_models/best_model.pkl
  ✓ Found: stress_models/scaler.pkl
  ✓ Found: stress_models/features.pkl

1.2 Loading and testing model...
  ✓ Model loaded successfully
  ✓ Scaler loaded successfully
  ✓ Features loaded: ['heart_rate', 'hrv', 'spo2', 'systolic_bp', 'diastolic_bp', 'body_temp', 'sleep_hours', 'blink_rate']

  ✓ Model test successful!
    Test prediction: 0
    Test probabilities: [9.99936250e-01 2.38976209e-05 3.98526175e-05]


In [11]:
print("\n" + "="*80)
print("PART 2: CHECKING MODEL ACCURACY")
print("="*80)

def check_model_accuracy():
    """Evaluate model performance on test data"""
    
    print("\n2.1 Loading test dataset...")
    
    # Check if dataset exists
    if not os.path.exists('stress_detection_realistic_1500_rows.csv'):
        print("  ⚠️ Original dataset not found - skipping accuracy check")
        return
    
    # Load data
    df = pd.read_csv('stress_detection_realistic_1500_rows.csv')
    X = df.drop('stress_level', axis=1)
    y = df['stress_level']
    
    print(f"  ✓ Loaded {len(df)} samples")
    
    # Load model and scaler
    with open('stress_models/best_model.pkl', 'rb') as f:
        model = pickle.load(f)
    with open('stress_models/scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)
    
    # Make predictions
    X_scaled = scaler.transform(X)
    predictions = model.predict(X_scaled)
    
    print("\n2.2 Model Performance Metrics:")
    print("-" * 80)
    
    # Overall accuracy
    accuracy = accuracy_score(y, predictions)
    print(f"\n  Overall Accuracy: {accuracy*100:.2f}%")
    
    if accuracy >= 0.90:
        print("  ✓ EXCELLENT - Model is very reliable!")
    elif accuracy >= 0.80:
        print("  ✓ GOOD - Model is reliable")
    elif accuracy >= 0.70:
        print("  ⚠️ FAIR - Model may need improvement")
    else:
        print("  ✗ POOR - Model needs retraining")
    
    # Per-class accuracy
    print("\n2.3 Per-Class Accuracy:")
    cm = confusion_matrix(y, predictions)
    
    for i, level_name in enumerate(['Low Stress', 'Medium Stress', 'High Stress']):
        correct = cm[i, i]
        total = cm[i, :].sum()
        class_acc = correct / total * 100 if total > 0 else 0
        print(f"  {level_name:15s}: {class_acc:5.1f}% ({correct}/{total} correct)")
        
        if class_acc >= 90:
            print(f"    ✓ Excellent for {level_name}")
        elif class_acc >= 80:
            print(f"    ✓ Good for {level_name}")
        else:
            print(f"    ⚠️ May miss some {level_name} cases")
    
    # Confusion Matrix
    print("\n2.4 Confusion Matrix:")
    print("    (Shows where model makes mistakes)")
    print("\n              Predicted")
    print("              Low    Med    High")
    for i, level in enumerate(['Low', 'Med', 'High']):
        print(f"  Actual {level:4s}  {cm[i,0]:4d}   {cm[i,1]:4d}   {cm[i,2]:4d}")
    
    print("\n2.5 Reliability Assessment:")
    
    # Check if model is balanced
    errors = []
    for i in range(3):
        for j in range(3):
            if i != j and cm[i, j] > 0:
                level_names = ['Low', 'Medium', 'High']
                errors.append(f"  • {cm[i,j]} {level_names[i]} cases predicted as {level_names[j]}")
    
    if len(errors) == 0:
        print("  ✓ PERFECT - No errors!")
    else:
        print("  Common mistakes:")
        for error in errors[:5]:  # Show top 5
            print(error)
    
    print("\n  ✓ Model verification complete!")

check_model_accuracy()


PART 2: CHECKING MODEL ACCURACY

2.1 Loading test dataset...
  ✓ Loaded 1500 samples

2.2 Model Performance Metrics:
--------------------------------------------------------------------------------

  Overall Accuracy: 95.40%
  ✓ EXCELLENT - Model is very reliable!

2.3 Per-Class Accuracy:
  Low Stress     :  98.4% (492/500 correct)
    ✓ Excellent for Low Stress
  Medium Stress  :  92.6% (463/500 correct)
    ✓ Excellent for Medium Stress
  High Stress    :  95.2% (476/500 correct)
    ✓ Excellent for High Stress

2.4 Confusion Matrix:
    (Shows where model makes mistakes)

              Predicted
              Low    Med    High
  Actual Low    492      8      0
  Actual Med     14    463     23
  Actual High     0     24    476

2.5 Reliability Assessment:
  Common mistakes:
  • 8 Low cases predicted as Medium
  • 14 Medium cases predicted as Low
  • 23 Medium cases predicted as High
  • 24 High cases predicted as Medium

  ✓ Model verification complete!


In [15]:
print("\n" + "="*80)
print("PART 3: USER INPUT PREDICTION SYSTEM")
print("="*80)

def predict_stress_with_details(heart_rate, hrv, spo2, systolic_bp, diastolic_bp, 
                                body_temp, sleep_hours, blink_rate, show_details=True):
    """
    Predict stress level with detailed explanation
    
    Parameters:
    -----------
    All 8 health parameters (see below)
    show_details: If True, shows detailed analysis
    
    Returns:
    --------
    Dictionary with prediction results
    """
    
    # Load model and scaler
    with open('stress_models/best_model.pkl', 'rb') as f:
        model = pickle.load(f)
    with open('stress_models/scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)
    
    # Prepare input
    features = [[heart_rate, hrv, spo2, systolic_bp, diastolic_bp, 
                 body_temp, sleep_hours, blink_rate]]
    features_scaled = scaler.transform(features)
    
    # Make prediction
    prediction = model.predict(features_scaled)[0]
    probabilities = model.predict_proba(features_scaled)[0]
    confidence = probabilities[prediction]
    
    # Results
    levels = ['Low', 'Medium', 'High']
    icons = ['🟢', '🟡', '🔴']
    
    result = {
        'level': int(prediction),
        'label': levels[prediction],
        'icon': icons[prediction],
        'confidence': float(confidence),
        'probabilities': {
            'Low': float(probabilities[0]),
            'Medium': float(probabilities[1]),
            'High': float(probabilities[2])
        },
        'input_data': {
            'heart_rate': heart_rate,
            'hrv': hrv,
            'spo2': spo2,
            'systolic_bp': systolic_bp,
            'diastolic_bp': diastolic_bp,
            'body_temp': body_temp,
            'sleep_hours': sleep_hours,
            'blink_rate': blink_rate
        }
    }
    
    if show_details:
        print("\n" + "="*70)
        print("PREDICTION RESULT")
        print("="*70)
        
        print("\nYour Input:")
        print(f"  Heart Rate: {heart_rate} BPM")
        print(f"  HRV: {hrv} ms")
        print(f"  SpO2: {spo2}%")
        print(f"  Blood Pressure: {systolic_bp}/{diastolic_bp} mmHg")
        print(f"  Body Temperature: {body_temp}°C")
        print(f"  Sleep: {sleep_hours} hours")
        print(f"  Blink Rate: {blink_rate} per minute")
        
        print("\n" + "-"*70)
        print("STRESS LEVEL PREDICTION")
        print("-"*70)
        
        print(f"\n  {result['icon']} STRESS LEVEL: {result['label'].upper()}")
        print(f"  Confidence: {result['confidence']:.1%}")
        
        print("\n  Probability Breakdown:")
        print(f"    Low Stress:    {result['probabilities']['Low']:.1%}")
        print(f"    Medium Stress: {result['probabilities']['Medium']:.1%}")
        print(f"    High Stress:   {result['probabilities']['High']:.1%}")
        
        # Interpretation
        print("\n" + "-"*70)
        print("INTERPRETATION")
        print("-"*70)
        
        if result['level'] == 0:  # Low
            print("\n  ✓ Your stress levels are HEALTHY")
            print("\n  What this means:")
            print("    • Your body is in a relaxed state")
            print("    • Vital signs are within normal range")
            print("    • No immediate health concerns")
            print("\n  Recommendations:")
            print("    • Keep up your current healthy habits")
            print("    • Maintain regular exercise")
            print("    • Continue good sleep routine")
            print("    • Stay socially connected")
            
        elif result['level'] == 1:  # Medium
            print("\n  ⚠️ Your stress levels are ELEVATED")
            print("\n  What this means:")
            print("    • Your body is showing signs of stress")
            print("    • Some vital signs are above normal")
            print("    • Early warning - take action now")
            print("\n  Recommendations:")
            print("    • Take regular breaks during work")
            print("    • Practice relaxation (deep breathing, meditation)")
            print("    • Ensure 7-8 hours of sleep")
            print("    • Reduce caffeine intake")
            print("    • Talk to someone about your concerns")
            print("    • Consider light exercise or yoga")
            
        else:  # High
            print("\n  🚨 Your stress levels are HIGH")
            print("\n  What this means:")
            print("    • Your body is under significant stress")
            print("    • Multiple vital signs are concerning")
            print("    • Immediate action recommended")
            print("\n  URGENT Recommendations:")
            print("    • Consult a healthcare professional SOON")
            print("    • Take time off if possible")
            print("    • Prioritize rest and sleep")
            print("    • Avoid additional stressors")
            print("    • Reach out to support network")
            print("    • Practice stress management techniques")
            print("    • Monitor your symptoms closely")
        
        # Confidence interpretation
        print("\n" + "-"*70)
        print("CONFIDENCE LEVEL")
        print("-"*70)
        
        if result['confidence'] >= 0.90:
            print(f"\n  ✓ Very High Confidence ({result['confidence']:.1%})")
            print("    The model is very certain about this prediction.")
        elif result['confidence'] >= 0.75:
            print(f"\n  ✓ High Confidence ({result['confidence']:.1%})")
            print("    The model is confident about this prediction.")
        elif result['confidence'] >= 0.60:
            print(f"\n  ⚠️ Moderate Confidence ({result['confidence']:.1%})")
            print("    Your values are between stress levels.")
            print("    Consider retesting or monitoring symptoms.")
        else:
            print(f"\n  ⚠️ Low Confidence ({result['confidence']:.1%})")
            print("    Your values are borderline.")
            print("    Consider professional consultation.")
        
        print("\n" + "="*70)
    
    return result



PART 3: USER INPUT PREDICTION SYSTEM


In [17]:
print("\n" + "="*80)
print("PART 4: EXAMPLE PREDICTIONS")
print("="*80)

print("\nLet's test with 3 different cases to verify the model works correctly:")

# Example 1: Low Stress
print("\n" + "─"*80)
print("EXAMPLE 1: Healthy Person (Should be LOW STRESS)")
print("─"*80)

result1 = predict_stress_with_details(
    heart_rate=70,
    hrv=75,
    spo2=98,
    systolic_bp=110,
    diastolic_bp=75,
    body_temp=36.5,
    sleep_hours=8,
    blink_rate=15,
    show_details=True
)

# Verify
print("\n✓ Verification:")
if result1['level'] == 0:
    print("  ✓ CORRECT - Model correctly identified LOW stress")
else:
    print("  ✗ INCORRECT - Model should have predicted LOW stress")

# Example 2: Medium Stress
print("\n\n" + "─"*80)
print("EXAMPLE 2: Stressed Person (Should be MEDIUM STRESS)")
print("─"*80)

result2 = predict_stress_with_details(
    heart_rate=88,
    hrv=42,
    spo2=96,
    systolic_bp=128,
    diastolic_bp=84,
    body_temp=37.3,
    sleep_hours=5.5,
    blink_rate=26,
    show_details=True
)

print("\n✓ Verification:")
if result2['level'] == 1:
    print("  ✓ CORRECT - Model correctly identified MEDIUM stress")
else:
    print("  ✗ INCORRECT - Model should have predicted MEDIUM stress")

# Example 3: High Stress
print("\n\n" + "─"*80)
print("EXAMPLE 3: Highly Stressed Person (Should be HIGH STRESS)")
print("─"*80)

result3 = predict_stress_with_details(
    heart_rate=105,
    hrv=25,
    spo2=94,
    systolic_bp=145,
    diastolic_bp=92,
    body_temp=37.8,
    sleep_hours=3.5,
    blink_rate=35,
    show_details=True
)

print("\n✓ Verification:")
if result3['level'] == 2:
    print("  ✓ CORRECT - Model correctly identified HIGH stress")
else:
    print("  ✗ INCORRECT - Model should have predicted HIGH stress")


print("\n\n" + "─"*80)
print("EXAMPLE 3: Highly Stressed Person (Should be HIGH STRESS)")
print("─"*80)

result4 = predict_stress_with_details(
    heart_rate=65,
    hrv=50,
    spo2=98,
    systolic_bp=110,
    diastolic_bp=70,
    body_temp=35.8,
    sleep_hours=8,
    blink_rate=15,
    show_details=True
)

print("\n✓ Verification:")
if result4['level'] == 0:
    print("  ✓ CORRECT - Model correctly identified low stress")
else:
    print("  ✗ INCORRECT - Model should have predicted low stress")





PART 4: EXAMPLE PREDICTIONS

Let's test with 3 different cases to verify the model works correctly:

────────────────────────────────────────────────────────────────────────────────
EXAMPLE 1: Healthy Person (Should be LOW STRESS)
────────────────────────────────────────────────────────────────────────────────

PREDICTION RESULT

Your Input:
  Heart Rate: 70 BPM
  HRV: 75 ms
  SpO2: 98%
  Blood Pressure: 110/75 mmHg
  Body Temperature: 36.5°C
  Sleep: 8 hours
  Blink Rate: 15 per minute

----------------------------------------------------------------------
STRESS LEVEL PREDICTION
----------------------------------------------------------------------

  🟢 STRESS LEVEL: LOW
  Confidence: 100.0%

  Probability Breakdown:
    Low Stress:    100.0%
    Medium Stress: 0.0%
    High Stress:   0.0%

----------------------------------------------------------------------
INTERPRETATION
----------------------------------------------------------------------

  ✓ Your stress levels are HEALTHY

 

In [19]:
### print("\n\n" + "="*80)
print("PART 5: YOUR TURN - ENTER YOUR DATA")
print("="*80)

def interactive_prediction():
    """Get user input and make prediction"""
    
    print("\nEnter your health parameters to check your stress level:")
    print("(Press Enter to skip and use example values)")
    print("-"*70)
    
    try:
        # Get user input
        hr = input("\n1. Heart Rate (BPM) [e.g., 75]: ").strip()
        heart_rate = float(hr) if hr else 75.0
        
        hrv_input = input("2. Heart Rate Variability (ms) [e.g., 50]: ").strip()
        hrv = float(hrv_input) if hrv_input else 50.0
        
        spo2_input = input("3. SpO2 - Blood Oxygen (%) [e.g., 97]: ").strip()
        spo2 = float(spo2_input) if spo2_input else 97.0
        
        sys_input = input("4. Systolic Blood Pressure (mmHg) [e.g., 120]: ").strip()
        systolic_bp = float(sys_input) if sys_input else 120.0
        
        dia_input = input("5. Diastolic Blood Pressure (mmHg) [e.g., 80]: ").strip()
        diastolic_bp = float(dia_input) if dia_input else 80.0
        
        temp_input = input("6. Body Temperature (°C) [e.g., 36.8]: ").strip()
        body_temp = float(temp_input) if temp_input else 36.8
        
        sleep_input = input("7. Sleep Hours [e.g., 7]: ").strip()
        sleep_hours = float(sleep_input) if sleep_input else 7.0
        
        blink_input = input("8. Eye Blink Rate (per minute) [e.g., 20]: ").strip()
        blink_rate = float(blink_input) if blink_input else 20.0
        
        # Make prediction
        result = predict_stress_with_details(
            heart_rate, hrv, spo2, systolic_bp, diastolic_bp,
            body_temp, sleep_hours, blink_rate,
            show_details=True
        )
        
        return result
        
    except ValueError as e:
        print(f"\n❌ Error: Please enter valid numbers")
        return None
    except Exception as e:
        print(f"\n❌ Error: {e}")
        return None

# Prompt user
print("\nWould you like to test with YOUR data?")
user_choice = input("Enter 'yes' to input your data, or any key to skip: ").strip().lower()

if user_choice in ['yes', 'y']:
    user_result = interactive_prediction()
    
    if user_result:
        print("\n" + "="*80)
        print("THANK YOU FOR USING THE STRESS DETECTION SYSTEM!")
        print("="*80)
        print("\nRemember:")
        print("  • This is a predictive tool, not a medical diagnosis")
        print("  • If concerned about your health, consult a doctor")
        print("  • Track your stress over time for better insights")
else:
    print("\nSkipped interactive input.")

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*80)
print("VERIFICATION COMPLETE!")
print("="*80)


PART 5: YOUR TURN - ENTER YOUR DATA

Would you like to test with YOUR data?


Enter 'yes' to input your data, or any key to skip:  yes



Enter your health parameters to check your stress level:
(Press Enter to skip and use example values)
----------------------------------------------------------------------



1. Heart Rate (BPM) [e.g., 75]:  75
2. Heart Rate Variability (ms) [e.g., 50]:  50
3. SpO2 - Blood Oxygen (%) [e.g., 97]:  98
4. Systolic Blood Pressure (mmHg) [e.g., 120]:  125
5. Diastolic Blood Pressure (mmHg) [e.g., 80]:  85
6. Body Temperature (°C) [e.g., 36.8]:  37
7. Sleep Hours [e.g., 7]:  7
8. Eye Blink Rate (per minute) [e.g., 20]:  23



PREDICTION RESULT

Your Input:
  Heart Rate: 75.0 BPM
  HRV: 50.0 ms
  SpO2: 98.0%
  Blood Pressure: 125.0/85.0 mmHg
  Body Temperature: 37.0°C
  Sleep: 7.0 hours
  Blink Rate: 23.0 per minute

----------------------------------------------------------------------
STRESS LEVEL PREDICTION
----------------------------------------------------------------------

  🟢 STRESS LEVEL: LOW
  Confidence: 57.8%

  Probability Breakdown:
    Low Stress:    57.8%
    Medium Stress: 42.1%
    High Stress:   0.1%

----------------------------------------------------------------------
INTERPRETATION
----------------------------------------------------------------------

  ✓ Your stress levels are HEALTHY

  What this means:
    • Your body is in a relaxed state
    • Vital signs are within normal range
    • No immediate health concerns

  Recommendations:
    • Keep up your current healthy habits
    • Maintain regular exercise
    • Continue good sleep routine
    • Stay socially connected

--------